# 编程入门13：Python文本处理
计算机经常需要对文本进行各种操作，我们知道Python语言表示文本的类型是字符串（str），其中可以包含任何字符。系统内部以标准的“统一码”（Unicode）来处理所有字符，用ord()函数可以返回一个字符对应的“码位”，用chr()函数则可以返回码位对应的字符。Unicode码位的取值范围是0x0至0x10ffff——这是十六进制数，等于十进制数0到1114111，例如最近加入Unicode字符集的“颜文字”（Emoji）位于0x1f300~0x1f6ff区域。
```
In [1]: ord("A")
Out[1]: 65

In [2]: chr(65)
Out[2]: 'A'

In [3]: ord("一")
Out[3]: 19968

In [4]: chr(19968)
Out[4]: '一'

In [5]: for i in range(0x1f600, 0x1f650):  # 基本表情类颜文字
   ...:     print(chr(i), end="")
   ...:     if i % 16 == 15:
   ...:         print()
   ...:         
😀😁😂😃😄😅😆😇😈😉😊😋😌😍😎😏
😐😑😒😓😔😕😖😗😘😙😚😛😜😝😞😟
😠😡😢😣😤😥😦😧😨😩😪😫😬😭😮😯
😰😱😲😳😴😵😶😷😸😹😺😻😼😽😾😿
🙀🙁🙂🙃🙄🙅🙆🙇🙈🙉🙊🙋🙌🙍🙎🙏
```

你还可以在字符串中使用反斜杠打头的“转义序列”（Escape Sequence）来表示字符，例如反斜杠加n表示换行符，而反斜杠加手动换行则会被解释器完全忽略。以下是常用的转义序列：
- \换行：忽略
- \\\\：反斜杠（\）
- \\'：单引号（'）
- \\"：双引号（"）
- \a：响铃（BEL）
- \n：换行（LF）
- \r：回车（CR）
- \t：水平制表符（TAB）
- \xhh：十六进制数hh码位的字符，即扩展西文字符（ASCII）
- \uhhhh：十六进制数hhhh码位的字符，即基本多语言位面字符（BMP）
- \Uhhhhhhhh：十六进制数hhhhhhhh码位的字符，即任何字符

以下是在交互模式下演示这些转义序列：
```
In [6]: print("人人为我\
   ...: 我为人人")
人人为我我为人人

In [7]: print("人人为我\n我为人人")
人人为我
我为人人

In [8]: print("C:\\Windows\\System32")  # Windows路径中的\需要转义
C:\Windows\System32

In [9]: r"C:\Windows\System32"  # 加r前缀表示“原始字符串”，解释器会自动转义
Out[9]: 'C:\\Windows\\System32'

In [10]: print("\x07\u4e00\U0001f600")  # 响铃，汉字一，颜文字笑脸
一😀
```

实际上反斜杠加手动换行也可以在字符串之外使用，当某条语句长度超过80字符时，你可以用这个方法来分行，以使代码格式符合PEP8规范，更为易读。

计算机在保存和传输数据时都是以8个二进制位组成的“字节”作为基本单位的，对于文本来说就需要规定某种字符“编码”（Encoding），通过只有256个码位的字节序列来表示任何字符。Python语言的字节序列类型是加b前缀表示的“字节串”（bytes），其中的元素显示为基本西文字符或\xhh转义序列。用str.encode()方法可以将str编码为bytes，用bytes.decode()方法可以将bytes解码为str。Python默认使用“8位通用转换格式”（UTF-8）进行文本编码和解码，中文常用的字符编码格式还有“国标码扩展”（GBK）等：
```
In [11]: "ABC一二三".encode()  # 使用默认的UTF-8编码
Out[11]: b'ABC\xe4\xb8\x80\xe4\xba\x8c\xe4\xb8\x89'

In [12]: "ABC一二三".encode("gbk")  # 使用GBK编码
Out[12]: b'ABC\xd2\xbb\xb6\xfe\xc8\xfd'

In [13]: b'ABC\xe4\xb8\x80\xe4\xba\x8c\xe4\xb8\x89'.decode()  # 使用默认的UTF-8解码
Out[13]: 'ABC一二三'

In [14]: b'ABC\xd2\xbb\xb6\xfe\xc8\xfd'.decode("gbk")  # 使用GBK解码
Out[14]: 'ABC一二三'

In [15]: len(b'ABC\xd2\xbb\xb6\xfe\xc8\xfd')  # 查看字节串的长度
Out[15]: 9
```
通过查看字节串的长度，可以看到对于每个汉字UTF-8使用三字节编码，GBK则使用两字节编码。如果编码和解码使用的格式不匹配，就会出现“乱码”——请访问以下链接查看完整的字符编码列表：https://docs.python.org/3/library/codecs.html#standard-encodings

在不同系统之间传递数据时，需要先把数据对象转化为通用格式再恢复回来，这个过程称为“序列化”（Serialize）和“反序列化”（Unserialize）。JSON（JavaScript Object Notation）就是一种基于文本的通用数据交换格式，在互联网上应用非常广泛。要在Python中处理JSON，可以引入标准库的json模块。其中dumps()函数将Python对象序列化为JSON字符串，loads()函数将JSON字符串反序列化为Python对象：
```
In [16]: import json

In [17]: dictdata = {"baidu": "百度", "alibaba": "阿里巴巴", "tencent": "腾讯"}

In [18]: jsondata = json.dumps(dictdata, ensure_ascii=False)  # ensure_ascii参数指定非ASCII字符是否要转义

In [19]: jsondata
Out[19]: '{"baidu": "百度", "alibaba": "阿里巴巴", "tencent": "腾讯"}'

In [20]: json.loads(jsondata)
Out[20]: {'baidu': '百度', 'alibaba': '阿里巴巴', 'tencent': '腾讯'}
```

以下实例演示了“微历”天气API的使用——加参数访问网址返回GZIP压缩的JSON字节串，经过解压缩、解码和反序列化得到字典对象，即可输出其中的天气预报信息：
```
"""weather.py 获取天气预报信息
"""
from urllib.request import urlopen
from urllib.parse import quote
import gzip
import json
url = "http://wthrcdn.etouch.cn/weather_mini?city="
city = "拉萨"


def main():
    try:
        # 发起网络请求获取数据
        data = urlopen(url + quote(city)).read()
        # 解压缩并解码得到JSON格式字符串
        data_json = gzip.decompress(data).decode()
        # 反序列化为字典对象
        data_dict = json.loads(data_json)
        if data_dict.get("desc") != "OK":
            print("无法获取天气数据！")
        else:  # 输出日期、天气、气温信息
            print("天气预报：" + city)
            forecast = data_dict.get("data").get("forecast")
            for i in forecast:
                date = i.get("date")
                high = i.get("high")
                low = i.get("low")
                weather = i.get("type")
                print("{} {} {} ~ {}".format(date, weather, low, high))
    except Exception as ex:
        print(repr(ex))


if __name__ == "__main__":
    main()
```

——编程原来是这样……

## 编程小提示：QPython3
如果你想在便携设备上使用Python，可以安装QPython3这个App（注意是QPython3，另有一个QPython是基于Python2的），就能随时练习Python编程了（但不支持tkinter图形界面工具包）。更多详情可访问官方网站 http://www.qpython.org/
![13_qpython3.jpg](https://upload-images.jianshu.io/upload_images/10829283-f7d59dab194fe19d.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

下一篇：[编程入门14：Python模式匹配](14_pattern.ipynb)